# EE-411 Fundamentals of inference and learning

## Exercise session 13: Autoencoder, denoising and K-means

In the first part of this set of exercises we will write an Auto-encoder in Python using Keras, and we will apply it for a denoising problem, evaluating its performance in different situations. In the second part we will have a simple implementation of the K-means method for clustering using scikit-learn.

**What you will learn today:** In this notebook, we will use an alternative to Pytorch: Keras. Specifically, we will use it to implement an Autoencoder. Then, we will see how to do clustering with K-means using scikit-learn.


# 1) Autoencoder

An autoencoder, autoassociator or Diabolo network is an artificial neural network whose aim is to learn a representation (or an *encoding*) for a set of data, often for the purpose of dimensionality reduction. Recently, the autoencoder concept has become more widely used for denoising images, and we shall illustrate these in this notebook on the MNIST dataset. 

First, let us look download the images from the MNIST dataset, and look at some of them.

In [ ]:
from keras.datasets import mnist
import numpy as np
#We load the data
(x_train, _), (x_test, _) = mnist.load_data()
#We renormalize the data to a float in [0,1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
#We reshape the images to be 2d matrices of dimension n times d
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)
#We plot the first ten images
%matplotlib inline
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Let us define our autoencoder using Keras. If you already know how to work with Pytorch, this will be very intuitive. 

**First** we encode data by a first neural network that goes from the $784$ points to $128$, then $64$ and finally $32$. 

**Second** we write the decoder that performs the same operation, in reverse.

**Third** we can build our model and compile it,

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
input_img = Input(shape=(784,))
#Encoding
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
#Decoding
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)
#Putting all together
autoencoder_mlp = Model(input_img, decoded)
autoencoder_mlp.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_mlp.summary()

We now run the training for $100$ epochs. This takes some time because there are a lot of paramteres to learn. On a K80 GPU it takes 1sec/epoch, but on a standard CPU it is about 6sec/epoch.

In [ ]:
#Training the model -- CAN BE SKIPPED TO SAVE TIME
history_mlp = autoencoder_mlp.fit(x_train, x_train,
                epochs=500,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Plot history for accuracy
plt.plot(history_mlp.history['loss'])
plt.plot(history_mlp.history['val_loss'])
plt.title('model loss -- MLP autoencoder')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# SAVE/READ THE WEIGHTS FROM FILE
autoencoder_mlp.save_weights("dense_autoencoder.h5")
#autoencoder_mlp.load_weights("dense_autoencoder.h5")

Now, it is time to see the result of the learning process. 

Let us create an "encoder" model with the weights that we have learned, that outputs the result of the middle layer, and records the images corresponding to the test set observed after compression:

In [ ]:
encoder = Model(input_img, encoded)
encoded_imgs = encoder.predict(x_test)
print(encoded_imgs.shape)

We now do the same, but for the full autoencoder:

In [ ]:
decoded_imgs = autoencoder_mlp.predict(x_test)
print(decoded_imgs.shape)

Let us see how it looks:

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 6))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(8,4))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + n + n )
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
len(encoded_imgs)

This is quite nice, and indeed we see that we can recognize images quite easily. We can also have a look at the 32-dimensional encoded representations in the middle. Notice how the two "one" are coded similarly, as well as the two "four".

### Convolutional auto-encoder

Now we are going to build the autoencoer only with cnn:

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# After the encoding, the representation is (4, 4, 8) (128-dimensional)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder_cnn = Model(input_img, decoded)
autoencoder_cnn.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_cnn.summary()

Since we are using cnn layers, images should be kept as two-dimensional arrays. Once this is done, we repeat the fitting operation. It takes about 5s by epoch on a K80 GPU.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

##### 1) Train the model as before (If it takes too much time, use the cnn_autoencoder.h5 file in the repo)

In [ ]:
### YOUR CODE

##### 2) Plot history for accuracy

In [ ]:
### YOUR CODE

##### 3) Display the reconstructed images and compare them to the original ones. Display also the encoded representations as before

In [ ]:
### YOUR CODE

## Denoising with auto-encoders

### Noisy images

Now, we shall use our auto-encoders for denoising. We will train the autoencoder to map noisy images to clean ones. This will be done by applying a gaussian noise (and clip the images between 0 and 1) to the training set.

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Let us see how these looks:

In [ ]:
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Dense denoiser

##### 4) Using the simple dense network we defined above, train it on the noisy data

In [ ]:
### YOUR CODE

##### 5) Plot history for accuracy. Then, compare the reconstructed images to the original (noisy) ones.

In [ ]:
### YOUR CODE

### CNN denoiser

In order to improve the quality of the reconstruction, we shall use a CNN with more filters per layer...

In [ ]:
# Reshape the images to use a CNN
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  
x_train_noisy = np.reshape(x_train_noisy, (len(x_train_noisy), 28, 28, 1))  
x_test_noisy = np.reshape(x_test_noisy, (len(x_test_noisy), 28, 28, 1))  

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Here this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

denoiser_cnn = Model(input_img, decoded)
denoiser_cnn.compile(optimizer='adam', loss='binary_crossentropy')
denoiser_cnn.summary()

... and train it a bit longer (8sec/epoch on a GPU).

##### 5) Plot history for accuracy. Then, compare the reconstructed images to the original (noisy) ones.

In [ ]:
### YOUR CODE

# 2) K-means

Now we come back to scikit-learn to see how to easily implement K-means 

*1. Generate K Gaussian clusters of $n$ samples. Each cluster has $n_k = n/K$ data points $\boldsymbol{x}^{\mu}_{i,k} \in \mathbb{R}^{d}$ of mean  $\mu_k \in \mathbb{R}^{d}$, and covariance matrix  $\Sigma = \sigma \mathbb{I}_{d\times d}$. Take $d = 2$. Plot the data points in the 2-dimensional space with their centroids and assign to each data point a color according to cluster membership.*


Hint 1: for example, you can choose $n_k=100$, $K=3$ and $\sigma=1$. Try fixing some well-distanced centroids for this first part.

In [ ]:
#Hint 2: Use the following function to get the distance of a point from a center

def distance_from_the_center(X, m):
  """
  This function assign to each example in X a label according to its distance from the center.
  Input:  X = data;
          m = centroids.
  Output: cluster_membership = array of length #of samples, containing the label reffering to the cluster to which examples in X belongs to;
          cost = sum of the square distances of each data point from its cluster 
  """

  cluster_membership = np.zeros(len(X)) # initialize an empty array of length = number of samples
  cost = 0. # initialize the cost function
  for mu in range(len(X)): # loop over all examples
    distances = np.zeros(K) # initialize an empty array of length = input dimension where to store all the distances a given example has from each centroid
    for k in range(K): # loop over all the clusters
      dist = np.linalg.norm(X[mu] - m[k]) # compute the square distance of an example mu from centroid k 
      distances[k] = dist # store the corresponding result in the array of distances
    cluster_membership[mu] = np.argmin(distances) # assign to the example a label referring to the closest cluster 
    cost += np.min(distances**2) # compute the cost function
  return cluster_membership, cost 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random

In [ ]:
### YOUR CODE

*2. Implement K-Means using the built-in methods from scikit-learn. You can use [this website](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) as a reference (by now you should be able to use scikit learn confidently). Plot the data points in the 2-dimensional space with their centroids and compare these to the predicted ones.*

Hint: You can use *Kmeans.cluster_centers_* and *Kmeans.labels_* to extract the means and the variances resepectively

In [ ]:
### YOUR CODE

*3. Vary the variance $\sigma$ of each cluster in $[0.1,100.]$ and plot the detected cluster again with their centroids and colors assigned according to cluster membership. In which way are the predictions affected when the noise level is higher?*

In [ ]:
### YOUR CODE